In [2]:
# %matplotlib inline
from utils.opconNosepokeFunctions import *


toneStartMarker = 23
trialStartMarker = 81
trialHitMarker = 51
trialMissMarker = 86
trialUnrewMarker = 87
sessionStartMarker = 61
rewardProbMarker = 83
trialProbMarker = 88
lickMarker = 22
list_sessionMarker = [12, 13]

<Figure size 640x480 with 0 Axes>

# load files

In [3]:
animal = 'Bidoof'
box = 10
date = '2026-01-28'
# end_date = '2026-01-05'
path = f"L:/4portProb/Box{box}/{animal}/"       # for any les animal
# path = f"L:/4portProb_ephys/Box2_ephys/{animal}/Behavior/"      # for ephys only

files = exclude_files(get_files(path))
set_cwd(path)
files = [file for file in files if (file >=f'{animal}-{date}*.dat')]#& (file <= f'{animal}-{end_date}*.dat')]

# files = [r"C:\Users\dlab\Documents\OpCon v2.2 Nosepoke\Bhindi-2025-02-27-15-38-26.dat"]
# set_cwd(r'C:\Users\dlab\Documents\OpCon v2.2 Nosepoke')
files.sort()
print(files)
list_sessionMarker = [13]

df = merge_files_to_df(files)

sessdf = trializer_v3(df, list_sessionMarker,
                      trialStartMarker, trialHitMarker,
                      trialMissMarker, sessionStartMarker,
                      trialProbMarker, rewardProbMarker,
                      lickMarker, animal)

# if you want to add block numbers, uncomment this section
# sessdf['sess_block'] = 0
# thing = sessdf.groupby(['animal', 'task', 'session']).datetime.head(1).astype('datetime64[ns]').diff().dt.seconds>2400
# sessdf.loc[thing[thing == True].index, 'sess_block'] = 1
# sessdf['sess_block'] = sessdf.groupby(['animal', 'task']).sess_block.cumsum()

['Bidoof-2026-01-28-10-50-16.dat', 'Bidoof-2026-01-29-10-50-16.dat', 'Bidoof-2026-01-30-10-50-16.dat', 'Bidoof-2026-01-31-10-50-16.dat', 'Bidoof-2026-02-01-10-50-16.dat', 'Bidoof-2026-02-02-10-50-16.dat']


## ignore this section, untested

In [3]:
def trializer_v4(df, list_sessionMarker,
                 trialStartMarker, trialHitMarker,
                 trialMissMarker, sessionStartMarker,
                 trialProbMarker, rewardProbMarker,
                 lickMarker, animal, arms = 4):
    
    ''' Function to convert a df of joined .dat files with column names in order :
    'event', 'value', 'sm', 'tp', 'smtime', 'eptime', 'dw', 'ww', 'gpio_lick'
    
    INPUTS: df, list of session state machine numbers, trial start marker, trial rewarded marker,
    trial miss/unrewarded marker, session start marker, trial probability marker, session reward prob marker, 
    lick marker, animal name, num arms in task
    
    OUTPUT: single trial-wise dataframe of any number of animals
    2024-11-22
    '''
    # initialize var
    sessnum = 0
    npcounter = 0
    sess_list = []
    rewprobl = []
    
    event_lookup = {
        trialHitMarker: 1,
        trialMissMarker: 0,
    }
    list_events = [trialStartMarker, trialHitMarker, trialMissMarker, trialProbMarker, sessionStartMarker,
                   trialProbMarker, rewardProbMarker, lickMarker]
    
    # filter events as np array to increase speed and avoid repetitive indexing
    filtered_events = df[(df['sm'].isin(list_sessionMarker)) & (df['event'].isin(list_events))].to_numpy()
    indices = np.where((filtered_events[:, 0] == trialStartMarker) | (filtered_events[:, 0] == sessionStartMarker) |
                       (filtered_events[:, 0] == rewardProbMarker))[0]
    
    for ind in indices:
        # extract information from the current trialStartMarker
        trialstart = filtered_events[ind, 4]
        port = filtered_events[ind, 1]
        datetime = filtered_events[ind, 5]
        task = filtered_events[ind, 2]
        event = filtered_events[ind, 0]
    
        # select events occurring after trialStartMarker
        nextEvents = filtered_events[ind:]
    
        try:
            # find the index of trialMissMarker or trialHitMarker in nextEvents
            status_indices = np.where((nextEvents[:, 0] == trialMissMarker) | (nextEvents[:, 0] == trialHitMarker))[0]
            lick_indices = np.where(((nextEvents[:, 0]) == lickMarker) & (nextEvents[:, 1] == 1) & (nextEvents[:, -8] == 1))[0]
            statusIndex = status_indices[0] if status_indices.size > 0 else None
            lickIndex = lick_indices[lick_indices<status_indices[1]] if status_indices.size > 1 else lick_indices
            
            ms_licktimes = str(nextEvents[lickIndex][:, 4]).strip('[]')
            ms_lastlick = nextEvents[lickIndex][:, 4][-1]
            s_licktimes = str(nextEvents[lickIndex][:, 5]).strip('[]')
    
            # find the index of trialProbMarker in nextEvents
            rewprob_index = np.where(nextEvents[:, 0] == trialProbMarker)[0][0] 
            #contLine: if np.any(nextEvents[:, 0] == trialProbMarker) else None
    
            if statusIndex is not None:
                # find reward, trialend, and rewprob based on identified events
                reward = event_lookup[nextEvents[statusIndex, 0]]
                trialend = nextEvents[statusIndex, 4]
                rewprob = nextEvents[rewprob_index, 1] if rewprob_index is not None else np.nan
                
            else:
                # if trialMissMarker or trialHitMarker not found, nan
                reward = np.nan
                trialend = np.nan
                rewprob = np.nan
                ms_lastlick = np.nan
                
        except IndexError:
            # set values to nan in case of issues
            reward = np.nan
            trialend = np.nan
            rewprob = np.nan
            ms_lastlick = np.nan
        rewprobl_str = str(np.array(rewprobl, dtype = int))
        
        # append to sess_list
        sess_list.append([npcounter, trialstart, port, reward, trialend, sessnum,
                          rewprob, datetime, task, event, rewprobl_str,
                          ms_licktimes, ms_lastlick, s_licktimes])
        npcounter += 1
    
        if filtered_events[ind, 0] == sessionStartMarker:
            # update session number when sessionStartMarker found
            rewprobl = []
            sessnum +=1
    
        if filtered_events[ind, 0] == rewardProbMarker:
            rewprobl.append(filtered_events[ind, 4])
    
    # create pandas dataframe from the list of all session data
    sessdf =  pd.DataFrame(sess_list, columns = ['trial', 'trialstart',
                                                 'port', 'reward', 'trialend',
                                                 'session', 'rewprob', 'eptime',
                                                 'task', 'event', 'rewprobfull',
                                                 'ms_licktimes', 'ms_lastlick', 's_licktimes'])
    
    # remove all lines corresponding to start of a session or reward prob marker
    sessdf = sessdf[(sessdf.event!=sessionStartMarker) & (sessdf.event!=rewardProbMarker)].reset_index(drop = True).drop(columns = 'event')
    
    # remove all lines that have (trialend-trialstart)> 5500 or negative (sum of tone + wait time)
    # is this check required?
    
    # reset trial numbers
    sessdf['trial'] = np.arange(len(sessdf))
    
    # add animal information
    sessdf['animal'] = animal
    
    # add datetime
    sessdf['datetime'] = pd.to_datetime(sessdf.eptime, unit='s')

    return sessdf

In [9]:
### exclude files if required, by expression
# default: exclude_files(files, exclusion=['Roselia.+.dat', 'test-2022.+.dat', 'test-2023.+.dat', 'Azurill.+.dat'])
# animal = 'Bayleef'
# box = 7
# date = '2024-08-04'

# files = exclude_files(get_files(f"L:/4portProb/Box{box}/{animal}/"))
# files = [file for file in files if (file >=f'{animal}-{date}*.dat')]
# files.sort()
# list_sessionMarker = [12, 13]

# df = merge_files_to_df(files)
# sessdf = trializer_v3(df, list_sessionMarker,
#                       trialStartMarker, trialHitMarker,
#                       trialMissMarker, sessionStartMarker,
#                       trialProbMarker, rewardProbMarker, animal)
sessdf.task = sessdf.task.replace({13:'unstr', 12:'str'})
# reducing filesize on disk
sessdf["animal"] = sessdf["animal"].astype("category")
sessdf["task"] = sessdf['task'].astype("category")
# sessdf["rewprobfull"] = sessdf['rewprobfull'].astype(str).astype("category")

In [9]:
# check which sms on this day
stateMachines = np.sort(df['sm'].unique())
print(stateMachines)
SMlist = ['DailyWater0', 'WeeklyWater1', 'HouseLightsOn2', 'HouseLightsOff3', 'HouseLightsOnToneWater4',
           'BlinkLightsToneWater5', 'NosepokeNonSession6', 'LickTest7', 'LickTrain8', 'Nosepoke2sound9','Nosepoke1sound10',
          'NosepokeProb7011', 'NosepokeStr12', 'NosepokeUNStr13', 'NosepokeProb4014','ReadSessionParams15','NosepokeNonSession216','DoNothing17']
# session_rew_daily=(max(df[(df[2]<11) & (df[2]>6)][6]) - min(df[(df[2]<11) & (df[2]>6)][6]))
# non_session_rew_daily=max(df[df[2].isin([6,14])][6])-min(df[df[2].isin([6,14])][6])
# print('max day reward from sessions =', session_rew_daily)
# print('max day reward from non-session =',non_session_rew_daily)
# print('total reward through np =', session_rew_daily+non_session_rew_daily)

[ 0  1  2  3  4  5  7  9 10 11 13 17]


In [6]:
sessdf

,trial,trialstart,port,reward,trialend,session,rewprob,eptime,task,rewprobfull,animal,datetime
0,0,1646.0,3.0,1.0,2497.0,1,80.0,1.768173e+09,13.0,[15 46 80 10],22,2026-01-11 23:16:26
1,1,3668.0,2.0,0.0,4349.0,1,46.0,1.768173e+09,13.0,[15 46 80 10],22,2026-01-11 23:16:28
2,2,4805.0,2.0,1.0,5581.0,1,46.0,1.768173e+09,13.0,[15 46 80 10],22,2026-01-11 23:16:29
3,3,6742.0,3.0,0.0,7373.0,1,80.0,1.768173e+09,13.0,[15 46 80 10],22,2026-01-11 23:16:31
4,4,8061.0,2.0,1.0,8668.0,1,46.0,1.768173e+09,13.0,[15 46 80 10],22,2026-01-11 23:16:32
...,...,...,...,...,...,...,...,...,...,...,...,...
859,859,431428.0,3.0,0.0,432614.0,6,15.0,1.768189e+09,13.0,[10 80 15 46],22,2026-01-12 03:38:20
860,860,434170.0,3.0,0.0,436212.0,6,15.0,1.768189e+09,13.0,[10 80 15 46],22,2026-01-12 03:38:23
861,861,443964.0,3.0,1.0,445319.0,6,15.0,1.768189e+09,13.0,[10 80 15 46],22,2026-01-12 03:38:33
862,862,451383.0,2.0,1.0,453600.0,6,80.0,1.768189e+09,13.0,[10 80 15 46],22,2026-01-12 03:38:40


# Nosepoke session analysis

In [4]:
# %matplotlib nbagg
# this plots blockwise raster of choice
%matplotlib qt

import ipywidgets as widgets
from IPython.display import display
from matplotlib.patches import Rectangle
from utils.plotSettings import *
# import matplotlib.pyplot as plt
import seaborn as sns


# Create a function to update the plot based on user input
def update_plot(session_range):
    sessdf_filtered = sessdf[sessdf['session'].between(*session_range)]
    markers = {0: "$o$", 1: "o"}   # for reward-wise raster, useful for initial stages or zooming into data
#     markers = {0: 2, 1:3}
#     fig = plt.figure(figsize=(15, 8))
    g = sns.relplot(data=sessdf_filtered, y='port', x='trial', 
                    hue='rewprob',
                    aspect=11.7/5.5,
                    linewidth=0.75, s=100, palette = 'Blues', marker = '|', # change '|' to markers for reward-wise raster
                    # markers=markers, style='reward',
                    hue_norm=(-20, 80))
    axes = g.axes.flatten()
    last = sessdf.groupby('session')['trial'].max().to_list()
    start = sessdf.groupby('session')['trial'].min().to_list()
    highport = [ind+1 for i in sessdf.groupby('session').head(1).rewprobfull for ind, j in enumerate(i.strip('[]').split()) if j=='80']
    for ax in axes:
        for ind, l in enumerate(last):
            ax.axvline(l, linewidth=1, color='grey')
#             ax.axvline(, linewidth = 1, color = 'red')
            ax.set_yticks([1, 2, 3, 4])
            ax.add_patch(Rectangle((start[ind]-1, highport[ind]-0.5), l-start[ind], 1, fc = 'xkcd:blue', alpha = 0.1))
    
    plt.ylim(0.5, 4.5)
    plt.title('Choices', y=1.05)
    g._legend.set_title('Rew. %')
    # g._legend.texts[0].set_text("Reward %")
    # g._legend.texts[5].set_text("Outcome")
    plt.xticks(ticks = [], labels = [])

    plt.xlabel(' ', labelpad=20)
    plt.ylabel('Port')
    
    # Calculate x-axis limits based on the displayed sessions
    x_limit = (sessdf_filtered['trial'].min(), sessdf_filtered['trial'].max())
    plt.xlim(*x_limit)
    sns.despine(bottom = True)
#     plt.savefig('C:/Users/dlab/OneDrive - Indian Institute of Science/Drawings/plots/15012024/eg.svg', dpi = 600)
    plt.show()

# Define the session range using a widget
session_range_slider = widgets.IntRangeSlider(
    value=[sessdf['session'].min(), sessdf['session'].max()],
    min=sessdf['session'].min(),
    max=sessdf['session'].max(),
    step=1,
    description='Session Range:',
    continuous_update=False
)

# Create an interactive plot using interactive
interactive_plot = widgets.interactive(update_plot, session_range=session_range_slider)

# Display the interactive plot
display(interactive_plot)
plt.xticks(ticks = [], labels = [])
# plt.tight_layout()
# sns.set_context('talk')
# plt.title('Blissey - DLS then DMS lesion')

interactive(children=(IntRangeSlider(value=(1, 45), continuous_update=False, description='Session Range:', max…

([], [])

In [12]:
# check if there are any duplicated trials - this means there's something wrong with the box
mask = sessdf.duplicated(subset = ['animal', 'session', 'trialstart', 'trialend', 'eptime'], keep = False)
sessdf[mask]

,trial,trialstart,port,reward,trialend,session,rewprob,eptime,task,rewprobfull,animal,datetime


In [5]:
#plot number of times each port was chosen/total trials in block as a function of rewprob at that port, for all trials indf; average across blocks
port_choices = [1, 2, 3, 4]
#group by unique rewprob values, find unique rewprob values across all ports first
colors_ports = ['red', 'green', 'blue', 'purple']
unique_rewprobs = np.unique(sessdf['rewprob'])
# print(unique_rewprobs)
#remove nan from unique_rewprobs
unique_rewprobs = unique_rewprobs[~np.isnan(unique_rewprobs)]
print(unique_rewprobs)
#remove rows with nan rewprob
sessdf = sessdf[~sessdf['rewprob'].isna()]
#for every block, calculate number of times each port was chosen/total trials in block for each unique rewprob value
nblocks = sessdf['session'].nunique()
port_choice_matrix = np.zeros((len(unique_rewprobs), len(port_choices), nblocks))
#for all blocks
#save all rewprobfull per block in a list
for block in range(1, sessdf['session'].nunique()+1):
    block_data = sessdf[sessdf['session'] == block]
    #for all ports
    for p, port in enumerate(port_choices):
        port_data = block_data[block_data['port'] == port]
        #for all unique rewprobs
        for r, rewprob in enumerate(unique_rewprobs):
            n_choices = len(port_data[port_data['rewprob'] == rewprob])
            port_choice_matrix[r, p, block-1] = n_choices

bias_matrix = np.zeros((port_choice_matrix.shape[0], port_choice_matrix.shape[1]))
counter = np.zeros((port_choice_matrix.shape[0], port_choice_matrix.shape[1]))

for p in range(port_choice_matrix.shape[1]):
    for r in range(port_choice_matrix.shape[0]):
        bias_matrix[r, p] = 0
        for b in range(port_choice_matrix.shape[2]):
            ntrls = np.sum(port_choice_matrix[:,:,b])
            if ntrls != 0:
                bias_matrix[r, p] += port_choice_matrix[r, p, b] / ntrls
            if port_choice_matrix[r, p, b] > 0:
                counter[r, p] += 1

#divide fin_mat by counter
for p in range(port_choice_matrix.shape[1]):
    for r in range(port_choice_matrix.shape[0]):
        if counter[r, p] > 0:
            bias_matrix[r, p] = bias_matrix[r, p] / counter[r, p]
print('Bias matrix:', bias_matrix)


#plot as line plots for each port
plt.figure(figsize=(7, 6))
for p, port in enumerate(port_choices):
    plt.plot([0,1,2,3], bias_matrix[:, p], label=f'Port {port}', marker='o', linewidth=2, color=colors_ports[p])
plt.xlabel('Reward Probability')
plt.xticks([0,1,2,3], labels=[int(r) for r in unique_rewprobs])
plt.ylabel(f'Fraction chosen (averaged across {nblocks} blocks)')
plt.title(f'{animal}')
plt.legend()
plt.show()

n_rows, n_cols = bias_matrix.shape
rpf = [10,15,46,80]
labels = [i+1 for i in range(n_cols)]

# Angles for radar
angles = np.linspace(0, 2*np.pi, n_cols, endpoint=False)
angles = np.concatenate([angles, [angles[0]]])  # close loop

fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))
for i in range(n_rows):
    values = bias_matrix[i]
    values = np.concatenate([values, [values[0]]])  # close loop

    ax.plot(angles, values, linewidth=2, label=f'prob {rpf[i]}', color=colors_ports[i])
    # ax.fill(angles, values, alpha=0.15)

ax.set_thetagrids(np.degrees(angles[:-1]), labels)
#reduce the yticks to 3
# ax.set_yticks([])
ax.legend(loc='upper right', bbox_to_anchor=(1.25, 1.1))
plt.show()
 

[10. 15. 46. 80.]
Bias matrix: [[0.23992842 0.21664039 0.06548884 0.08172877]
 [0.01884125 0.15264905 0.06221155 0.18157653]
 [0.09567689 0.38050731 0.15633088 0.34305274]
 [0.29125086 0.656233   0.51669334 0.78090798]]


In [36]:
# add columns to make transition matrix
sessdf['choice_t1'] = sessdf.groupby(['animal','session']).port.shift(-1)
sessdf['choice_t2'] = sessdf.groupby(['animal','session']).port.shift(-2)
sessdf['shift_t0'] = (sessdf['choice_t1']==sessdf['port']).replace({True: 0, False: 1})
sessdf['shift_t1'] = (sessdf['choice_t2']==sessdf['port']).replace({True: 0, False: 1})

C:\Users\dlab\AppData\Local\Temp\ipykernel_18888\2834681967.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  sessdf['shift_t0'] = (sessdf['choice_t1']==sessdf['port']).replace({True: 0, False: 1})
C:\Users\dlab\AppData\Local\Temp\ipykernel_18888\2834681967.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  sessdf['shift_t1'] = (sessdf['choice_t2']==sessdf['port']).replace({True: 0, False: 1})


In [29]:
import matplotlib.pyplot as plt
plt.figure()
set_cwd('C:/Users/dlab/rishika_sim')
from utils.plotSettings import *
parula = get_parula_cmap()

ax = plt.subplot(111)
ratdf = sessdf[ ~(sessdf.duplicated(subset = ['animal', 'session', 'trialstart', 'trialend', 'eptime'], keep = False))]
# raltzdf = raltzdf.groupby('session').filter(lambda x: x.reward.size >= trialsinsess).groupby('session').head(trialsinsess)
sns.heatmap(pd.crosstab(ratdf.port, ratdf.choice_t1, normalize = 'index'), 
            ax = ax,
            cmap = parula,
            annot = True,
            fmt = '.2f',
            vmin = 0.0,
            vmax = 0.1,
            square = True,
            # mask = np.eye(4),
            xticklabels = np.arange(1,5), yticklabels = np.arange(1,5))
# ax.patch.set_facecolor('white')
ax.set_xlabel('Choice at trial t+1', fontsize = 'x-large')
ax.set_ylabel('Choice at trial t', fontsize = 'x-large')
ax.tick_params(axis = u'both', which = u'both', length = 0, labelsize = 'large', color = 'grey')

In [ ]:
# plot reward rate / regret
window = 5
sessdf['rr'] = sessdf.groupby('session', as_index = False).reward.rolling(window, center=True).mean().reward
sessdf['regret'] = abs(sessdf['rewprob']-80)
ax= plt.subplot(111)
rr_mean_dict = {}
trialsinsess = 100
filtered = (sessdf.groupby('session')).filter(lambda x: x.reward.size >= trialsinsess)

# take only those sessions
filtered = filtered.groupby('session').head(trialsinsess)


# convert to numpy array
g = filtered.groupby('session').cumcount()
L = np.array(filtered.set_index(['session',g])
    .unstack(fill_value=0)
    .stack(future_stack = True).groupby(level=0)
    .apply(lambda x: x.rr.to_numpy())
    .to_numpy())

# mean
rr_mean = np.mean(L, axis=0)
# rr_mean_dict[animal] = rr_mean
ax.plot(rr_mean)#, label = i, c = colors[i])
sns.despine()
ax.set_xlabel('Trials')
ax.set_ylabel('Reward rate')
plt.tight_layout()

In [38]:
# bias analysis - this needs to be changed, a la last group meeting (normalize differently)
# set_cwd('/home/rishika/sim/')
# from supplementaryFunctions import *
%matplotlib qt
trialsinsess = 100
# new_unstr_sess_dict = loader('new_unstructured')
# sessdf = new_unstr_sess_dict['/home/rishika/sim/NAS_rishika/4portProb/Box2']
fig = plt.figure(figsize = (8,7))
ax = plt.subplot(111)
filtered = sessdf[~(sessdf.duplicated(subset = ['animal', 'session', 'trialstart', 'trialend', 'eptime'], keep = False))].groupby('session').filter(lambda x: x.reward.size >= trialsinsess)
# take only those sessions
filtered = filtered.groupby('session').head(trialsinsess)

# Precompute choice probabilities
session_sizes = filtered.groupby('session').size()
choice_data = []
for i in range(1, 5):
    port_data = filtered[filtered['port'] == i].groupby(['session', 'rewprob']).size().reset_index(name='count')
    port_data['choice_prob'] = port_data['count'] / port_data['session'].map(session_sizes)
    port_data['port'] = i
    choice_data.append(port_data)
choice_df = pd.concat(choice_data, ignore_index=True)

# Plot
for i in range(1, 5):
    port_df = choice_df[choice_df['port'] == i]
    sns.scatterplot(data=port_df, x='rewprob', y='choice_prob', alpha=0.1, ax=ax)
    mean_df = port_df.groupby('rewprob')['choice_prob'].mean().reset_index()
    sns.lineplot(data=mean_df, x='rewprob', y='choice_prob', ax=ax, linewidth=2, label=f'Port {i}')

ax.set_ylabel('choice probability')
ax.set_xlabel('reward percent')
sns.despine()
ax.legend()
plt.suptitle(f'Choice Probability vs. Reward Percent, per arm, animal = {animal}')
plt.tight_layout()
plt.show()

In [12]:
# reward collection latency
g = sns.relplot(data=sessdf, x=sessdf['trialend']-sessdf['trialstart'], y='trial', hue='reward')
axes = g.axes.flatten()

# find the last trial number of a session
last = sessdf.groupby('session')['trial'].max()
for ax in axes:
    for l in last:
        ax.axhline(l, linewidth = 0.7, color='grey')
        ax.set_xlabel('Latency (ms)')
        ax.set_xlim(0,5600)
        ax.set_ylim(bottom=-5)
sns.set_context('notebook')
plt.suptitle('Latency', y=1.05)
# plt.savefig('plots/25082023/dms_D03_latency.png', dpi = 300)

Text(0.5, 1.05, 'Latency')

In [201]:
# number of trials in each session
g = sns.relplot(data=sessdf['session'].value_counts(), kind='scatter', color='k')
g.set_xlabels('session')
g.set_ylabels('# of trials')
# for ax in g.axes.flat:
#     for i in sessdf['session']:
#         if i%3==0:
#             ax.axvline(x=i, color='xkcd:light grey', linestyle='-.')
plt.axhline(np.mean(sessdf['session'].value_counts()), c='r')
# plt.savefig('plots/25082023/dms_D03_numtrials.png', dpi = 300)

C:\Users\dlab\miniforge3\envs\rishika\lib\site-packages\seaborn\axisgrid.py:118: UserWarning: The figure layout has changed to tight
  self._figure.tight_layout(*args, **kwargs)


## old plots that may not mean anything

In [17]:
###### plot cumulative regret 
g = sns.relplot(data = sessdf, x = 'trial', y = np.cumsum(sessdf.regret), color = 'k', kind = 'line')
# find the last trial number of a session
last = sessdf.groupby('session')['trial'].max()
for ax in g.axes.flat:
    for l in last:
        ax.axvline(l, linewidth = 1, color='grey')
ax.set_ylabel('cumulative regret')

Text(-21.30555555555555, 0.5, 'cumulative regret')

In [18]:
# what is distance moved given last trial was not rewarded?
dist = []
dist_unrew = []
dist_rew = []
for i in range(len(sessdf)):
    if i+1<len(sessdf):
        if sessdf['reward'][i]==0:
            dist_unrew.append(sessdf['port'][i]-sessdf['port'][i+1])
        else:
            dist_rew.append(sessdf['port'][i]-sessdf['port'][i+1])
plt.plot(dist_unrew, 'k')
sns.despine()
# plt.plot(dist_rew, 'r')